# Create Submission File

## Imports and get data

In [ ]:
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import timedelta
import pandas as pd
import numpy as np
from HelpFunctions.calc_score import evaluate_horizon
from HelpFunctions.mix_models import mix_models
from HelpFunctions.date_and_time import most_recent_thursday, split_time

from DAX.HelpFunctions.get_dax_data import get_dax_data
from DAX.Models import baseline as dax_bl
from DAX.Models import garch11 as dax_garch

from Energy.HelpFunctions.get_energy_data import get_energy_data, prepare_data
from Energy.Models import baseline as en_bl
from Energy.Models import Model4_population as en_m4_pop
from datetime import datetime
import pytz

# import importlib
# importlib.reload(Energy.Models)

# import importlib
# importlib.reload(Energy.Models)

### DAX: Get and prepare Data
Use tuesday as last datapoint

In [ ]:
# Insert Date of Wednesday (forecast-day) here. The current day has to be wednesday!!
start_date = datetime(2023, 12, 20, 8, 0, 0)
assert start_date.weekday() == 2

In [ ]:
df_dax = get_dax_data()
# pretend_start_date = date(2023, 11, 22, 11, 00, 0, tzinfo=pytz.timezone('Europe/Berlin'))

df_dax = df_dax[df_dax.index.date <= start_date.date()]

In [ ]:
df_dax.tail()

### Energy: Get and prepare Data

In [ ]:
from Energy.HelpFunctions.get_energy_data import fetch_energy_data
fetch_energy_data()

In [ ]:
df_en = get_energy_data(force_return=True)
df_en = prepare_data(df_en)

In [ ]:
df_en = df_en.loc[df_en.index <= start_date]

In [ ]:
df_en.tail()

## Run models for the data

### DAX

In [ ]:
from DAX.Models import arma_garch_11 as ag
from DAX.Models import baseline as bl

import importlib
importlib.reload(ag)
importlib.reload(bl)


# dax_bl.baseline(df_dax)
# forecast_dax = dax_garch.garch11(df_dax)
forecast_dax = mix_models([ag.garch11_norm_1y, bl.baseline_1y], [1,1], df_dax, 'DAX')
forecast_dax


### Energy  

In [ ]:
from Energy.Models.baseline import baseline
from HelpFunctions.mix_models import mix_models_per_horizon
from Energy.Models.Model4_sunhours import model4_sunhours
from Energy.Models.Model4_holidays_2 import model4_holidays_2
from Energy.Models.Model4_population import model4_population
from Energy.Models.Model5 import model5

# en_bl.baseline(df_en, LAST_IDX=-1, offset_horizons=11)

weights_m5_bl_m4_x = [1,1,0,0,0]
functions_m5_bl_m4_x = [model5, baseline, model4_population, model4_holidays_2, model4_sunhours]




# forecast_en = en_m4_pop.model4_population(df_en, offset_horizons=15)
forecast_en = mix_models(functions_m5_bl_m4_x, weights_m5_bl_m4_x, df_en, target="energy", offset_horizons=15)
forecast_en

### Infections

In [ ]:
forecast_infections = {
    'forecast_date': [start_date.strftime('%Y-%m-%d')] * 5,
    'target': ['infections'] * 5,
    'horizon': ['0 week', '1 week', '2 week', '3 week', '4 week'],
    # 'q0.025': ['NA'] * 5,
    # 'q0.25': ['NA'] * 5,
    # 'q0.5': ['NA'] * 5,
    # 'q0.75': ['NA'] * 5,
    # 'q0.975': ['NA'] * 5,
    'q0.025': [np.nan] * 5,
    'q0.25': [np.nan] * 5,
    'q0.5': [np.nan] * 5,
    'q0.75': [np.nan] * 5,
    'q0.975': [np.nan] * 5
}

forecast_infections = pd.DataFrame(forecast_infections)

In [ ]:
forecast_infections

## Merge results

Check results: Are they valid, forecast_date is right? The last entry might be cut off

In [ ]:
forecasts = pd.concat([forecast_dax,forecast_en, forecast_infections]).reset_index().drop(columns=['index'])
forecasts

If everything seems good: Override forecast_date

In [ ]:
forecasts['forecast_date'] = [start_date.strftime('%Y-%m-%d') for _ in range(len(forecasts))]

In [ ]:
forecasts

### Check df & Save csv-File

In [ ]:
from HelpFunctions.check_submission import check_submission_df

check_submission_df(forecasts)
forecasts.to_csv(f'./Abgaben/{start_date.strftime("%Y%m%d")}_JakePeralta.csv', na_rep='NA', index=False)